# Dashboard Visualization of Science and Engineering PhDs awarded in the US

This is a test of the url for a sample dashboard from https://towardsdatascience.com/dash-for-beginners-create-interactive-python-dashboards-338bfcb6ffa4

http://127.0.0.1:8050/ 